# Synthetic Data Kit - Kendi Local LLM API Demo

Bu notebook, Meta'nın Synthetic Data Kit'ini **kendi local LLM API'niz** ile kullanarak sentetik veri oluşturmayı gösterir.

## ⚡ Temel Özellikler

- ✅ **vLLM gerekmez** - Kendi API endpoint'inizi kullanın
- ✅ **Script değil** - Kütüphane olarak kullanım
- ✅ **Dışarıdan veri** - Herhangi bir text/PDF/DOCX dosyasından veri oluşturma
- ✅ **4 adımlı pipeline** - Ingest → Create → Curate → Save

## 📋 İçindekiler

1. Kurulum
2. Kendi API'nizi Yapılandırma
3. Pratik Demo
4. Programmatik Kullanım (Python API)
5. Batch İşleme

---

## 1️⃣ Kurulum

İlk olarak kütüphaneyi pip ile kurun:

In [ ]:
# Synthetic Data Kit kurulumu
!pip install synthetic-data-kit

In [ ]:
# Gerekli kütüphaneler
import os
import json
import yaml
from pathlib import Path
from typing import Dict, List, Optional

print("✅ Import'lar başarılı!")

---

## 2️⃣ Kendi Local API'nizi Yapılandırma

**ÖNEMLİ:** Synthetic Data Kit, OpenAI-uyumlu API'lerle çalışır. 

Kendi local LLM API'niz (Ollama, LM Studio, Text Generation WebUI, vb.) OpenAI formatında endpoint sağlıyorsa direkt kullanabilirsiniz.

### 🔧 API Endpoint Ayarları

In [ ]:
# ⚙️ BURAYA KENDİ API BİLGİLERİNİZİ GİRİN

# Örnek: Ollama
# API_BASE = "http://localhost:11434/v1"
# MODEL_NAME = "llama3.1:8b"

# Örnek: LM Studio
# API_BASE = "http://localhost:1234/v1"
# MODEL_NAME = "llama-3.1-8b-instruct"

# Örnek: Text Generation WebUI (oobabooga)
# API_BASE = "http://localhost:5000/v1"
# MODEL_NAME = "your-model-name"

# Kendi API'niz (örnek)
API_BASE = "http://localhost:8000/v1"  # 👈 Kendi endpoint'iniz
MODEL_NAME = "llama-3.1-8b-instruct"   # 👈 Modelinizin adı
API_KEY = "sk-dummy"  # Bazı API'ler key gerektirmez, "dummy" bırakabilirsiniz

print(f"✅ API Ayarları:")
print(f"   Base URL: {API_BASE}")
print(f"   Model: {MODEL_NAME}")

### 🧪 API Bağlantı Testi

API'nizin çalıştığını test edelim:

In [ ]:
from openai import OpenAI

# OpenAI SDK ile kendi API'nize bağlanma
client = OpenAI(
    base_url=API_BASE,
    api_key=API_KEY
)

# Test mesajı
try:
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": "Merhaba! Kısa bir test mesajı."}
        ],
        max_tokens=50
    )
    
    print("✅ API bağlantısı başarılı!")
    print(f"\n🤖 Model yanıtı:\n{response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ API bağlantı hatası: {e}")
    print("\n💡 Kontrol edin:")
    print("   1. Local LLM sunucunuz çalışıyor mu?")
    print("   2. API_BASE URL'i doğru mu?")
    print("   3. MODEL_NAME doğru mu?")

### 📁 Çalışma Dizinlerini Oluştur

In [ ]:
# Proje dizin yapısı
BASE_DIR = Path("./synthetic_data_demo")
INPUT_DIR = BASE_DIR / "input"           # Kaynak dosyalar buraya
PARSED_DIR = BASE_DIR / "parsed"         # Parse edilmiş veriler
GENERATED_DIR = BASE_DIR / "generated"   # Oluşturulan QA'lar
CURATED_DIR = BASE_DIR / "curated"       # Filtrelenmiş veriler
CONFIG_DIR = BASE_DIR / "config"         # Konfigürasyon dosyaları

# Dizinleri oluştur
for dir_path in [INPUT_DIR, PARSED_DIR, GENERATED_DIR, CURATED_DIR, CONFIG_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("✅ Dizin yapısı oluşturuldu:")
print(f"   📂 {BASE_DIR}/")
print(f"      ├── input/        (Kaynak dosyalarınızı buraya koyun)")
print(f"      ├── parsed/       (Parse edilmiş chunk'lar)")
print(f"      ├── generated/    (Üretilen QA pair'leri)")
print(f"      ├── curated/      (Filtrelenmiş sonuçlar)")
print(f"      └── config/       (Yapılandırma dosyaları)")

### ⚙️ Config Dosyası Oluştur

Synthetic Data Kit için custom config dosyası:

In [ ]:
# Custom yapılandırma
config = {
    "llm": {
        "provider": "openai",  # OpenAI-uyumlu API kullanacağız
        "model": MODEL_NAME,
        "temperature": 0.7,
        "max_tokens": 2048
    },
    "openai": {
        "api_base": API_BASE,
        "api_key": API_KEY
    },
    "ingest": {
        "chunk_size": 2000,      # Text'i kaç karakterlik parçalara bölelim
        "chunk_overlap": 200     # Parçalar arasında overlap
    },
    "create": {
        "num_examples_per_chunk": 3,  # Her chunk'tan kaç QA üretilsin
        "format": "qa"                 # qa, cot, summary
    },
    "curate": {
        "quality_threshold": 7.0,  # 0-10 arası kalite skoru (7+ iyi)
        "enabled": True
    }
}

# Config dosyasını kaydet
config_file = CONFIG_DIR / "my_config.yaml"
with open(config_file, "w", encoding="utf-8") as f:
    yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

print(f"✅ Config dosyası oluşturuldu: {config_file}")
print("\n📄 İçerik:")
print("="*60)
print(yaml.dump(config, default_flow_style=False, allow_unicode=True))

---

## 3️⃣ Örnek Veri Hazırlama

Dışarıdan bir dosya yükleyebilir veya örnek bir text oluşturabilirsiniz:

In [ ]:
# Örnek metin oluştur (veya kendi dosyanızı yükleyin)
sample_text = """
Yapay Zeka ve Large Language Models

Yapay zeka (AI), makinelerin insan benzeri zeka gerektiren görevleri yerine getirmesini 
sağlayan bir teknolojidir. Son yıllarda, özellikle Large Language Models (LLM'ler) 
alanında büyük gelişmeler yaşandı.

LLM'ler, milyarlarca parametre içeren ve çok büyük metin veri setleri üzerinde eğitilen 
derin öğrenme modelleridir. GPT-4, Llama, Claude gibi modeller bu kategoriye girer.

Transformer Mimarisi

Modern LLM'lerin çoğu, 2017 yılında "Attention is All You Need" makalesinde tanıtılan 
Transformer mimarisini kullanır. Bu mimari, self-attention mekanizması sayesinde 
cümledeki kelimelerin birbirleriyle ilişkilerini etkili bir şekilde öğrenebilir.

Transformerlar, önceki RNN ve LSTM tabanlı modellere göre birçok avantaj sunar:
- Paralel işleme yeteneği
- Uzun menzilli bağımlılıkları yakalama
- Daha hızlı eğitim süresi

Fine-tuning ve Sentetik Veri

Fine-tuning, önceden eğitilmiş bir modelin belirli bir göreve veya domaine adapte 
edilmesi sürecidir. Bu işlem için kaliteli eğitim verisi gerekmektedir.

Sentetik veri, gerçek veri yerine yapay olarak üretilen veridir. LLM'leri kullanarak 
yüksek kaliteli QA (Soru-Cevap) çiftleri, Chain-of-Thought örnekleri veya özetler 
oluşturulabilir. Bu veriler fine-tuning için kullanılabilir.

Meta'nın Synthetic Data Kit'i gibi araçlar, bu süreci otomatikleştirerek kaliteli 
sentetik eğitim verisi oluşturmayı kolaylaştırır.

RAG (Retrieval Augmented Generation)

RAG, LLM'lerin güncel ve spesifik bilgilere erişmesini sağlayan bir tekniktir. 
Model, sorguya yanıt vermeden önce bir bilgi tabanından ilgili dokümanları alır 
ve bu bilgileri kullanarak yanıt üretir.

RAG sistemleri genellikle şu bileşenlerden oluşur:
1. Vektör veritabanı (embedding'ler için)
2. Retriever (ilgili dokümanları bulma)
3. Generator (LLM - yanıt oluşturma)
"""

# Dosyayı kaydet
input_file = INPUT_DIR / "ai_ml_tutorial.txt"
with open(input_file, "w", encoding="utf-8") as f:
    f.write(sample_text)

print(f"✅ Örnek dosya oluşturuldu: {input_file}")
print(f"📏 Dosya boyutu: {len(sample_text)} karakter")
print(f"\n💡 İpucu: Kendi dosyanızı kullanmak için '{INPUT_DIR}/' dizinine kopyalayın")

---

## 4️⃣ Synthetic Data Pipeline

### Pipeline Adımları:

1. **INGEST** → Belgeyi parse et ve chunk'lara böl
2. **CREATE** → Her chunk'tan QA pair'leri oluştur
3. **CURATE** → Kalite kontrolü yap, kötüleri filtrele
4. **SAVE-AS** → İstediğin formatta kaydet (Alpaca, ChatML, vb.)

### 🔸 Adım 1: INGEST - Belgeyi Parse Etme

In [ ]:
# INGEST komutu
!synthetic-data-kit ingest \
    {str(input_file)} \
    --config {str(config_file)}

print("\n✅ Ingest tamamlandı!")

In [ ]:
# Parse edilmiş dosyayı inceleyin
# Synthetic Data Kit genellikle data/parsed/ dizinine kaydeder
parsed_files = list(Path("data/parsed").glob("**/*.txt")) if Path("data/parsed").exists() else []

if parsed_files:
    print(f"📄 Parse edilmiş dosya: {parsed_files[0]}")
    with open(parsed_files[0], "r", encoding="utf-8") as f:
        content = f.read()
        print(f"📏 İçerik uzunluğu: {len(content)} karakter")
        print(f"\n📝 İlk 300 karakter:\n{content[:300]}...")
else:
    print("⚠️ Parse edilmiş dosya bulunamadı. Lütfen ingest komutunu kontrol edin.")

### 🔸 Adım 2: CREATE - QA Pair'leri Oluşturma

**ÖNEMLİ:** Bu adımda local LLM API'niz kullanılacak!

In [ ]:
# Parse edilmiş dosyayı bul
if parsed_files:
    parsed_file = parsed_files[0]
else:
    # Manuel olarak belirt
    parsed_file = Path("data/parsed/ai_ml_tutorial.txt")

# CREATE komutu - QA pair'leri oluştur
!synthetic-data-kit create \
    {str(parsed_file)} \
    --type qa \
    --num-pairs 10 \
    --config {str(config_file)}

print("\n✅ QA pair'leri oluşturuldu!")

In [ ]:
# Oluşturulan QA'ları incele
generated_files = list(Path("data/generated").glob("**/*qa_pairs.json")) if Path("data/generated").exists() else []

if generated_files:
    generated_file = generated_files[0]
    print(f"📄 Oluşturulan dosya: {generated_file}")
    
    with open(generated_file, "r", encoding="utf-8") as f:
        qa_data = json.load(f)
    
    print(f"\n📊 Toplam {len(qa_data)} QA pair oluşturuldu\n")
    print("="*80)
    
    # İlk 3 QA'yı göster
    for i, item in enumerate(qa_data[:3], 1):
        print(f"\n#{i}")
        print(f"❓ Soru: {item.get('question', 'N/A')}")
        print(f"✅ Cevap: {item.get('answer', 'N/A')}")
        print("-"*80)
else:
    print("⚠️ Oluşturulan dosya bulunamadı.")

### 🔸 Adım 3: CURATE - Kalite Kontrolü

LLM-as-a-judge kullanarak düşük kaliteli QA'ları filtreleyelim:

In [ ]:
if generated_files:
    # CURATE komutu
    !synthetic-data-kit curate \
        {str(generated_file)} \
        --threshold 7.0 \
        --config {str(config_file)}
    
    print("\n✅ Curate tamamlandı!")
else:
    print("⚠️ Önce CREATE adımını tamamlayın")

In [ ]:
# Curate edilmiş dosyayı incele
curated_files = list(Path("data/curated").glob("**/*_cleaned.json")) if Path("data/curated").exists() else []

if curated_files:
    curated_file = curated_files[0]
    print(f"📄 Filtrelenmiş dosya: {curated_file}")
    
    with open(curated_file, "r", encoding="utf-8") as f:
        curated_data = json.load(f)
    
    print(f"\n📊 Kalite kontrolü sonrası: {len(curated_data)} QA kaldı")
    print(f"🗑️ Filtrelenen: {len(qa_data) - len(curated_data)} QA")
    
    print("\n" + "="*80)
    for i, item in enumerate(curated_data[:2], 1):
        print(f"\n#{i}")
        if 'score' in item:
            print(f"⭐ Kalite Skoru: {item['score']}/10")
        print(f"❓ Soru: {item.get('question', 'N/A')}")
        print(f"✅ Cevap: {item.get('answer', 'N/A')[:200]}...")
        print("-"*80)
else:
    print("⚠️ Filtrelenmiş dosya bulunamadı.")

### 🔸 Adım 4: SAVE-AS - Format Dönüştürme

Fine-tuning için farklı formatlara dönüştürelim:

In [ ]:
if curated_files:
    output_alpaca = CURATED_DIR / "final_alpaca.json"
    
    # SAVE-AS: Alpaca formatı
    !synthetic-data-kit save-as \
        {str(curated_file)} \
        --format alpaca \
        --output {str(output_alpaca)}
    
    print(f"\n✅ Alpaca formatı: {output_alpaca}")
else:
    print("⚠️ Önce CURATE adımını tamamlayın")

In [ ]:
# Final dataset'i incele
if output_alpaca.exists():
    with open(output_alpaca, "r", encoding="utf-8") as f:
        final_data = json.load(f)
    
    print(f"📦 Final Dataset (Alpaca Format)")
    print(f"📊 Toplam örnek: {len(final_data)}\n")
    print("="*80)
    
    # İlk örneği göster
    print("\n📝 Örnek 1 (Alpaca format):")
    print(json.dumps(final_data[0], indent=2, ensure_ascii=False))
else:
    print("⚠️ Final dataset bulunamadı.")

---

## 5️⃣ Programmatik Kullanım (Python API)

CLI yerine doğrudan Python kodunda kullanmak isterseniz:

In [ ]:
# OpenAI SDK ile kendi QA generator yazma
from openai import OpenAI

def generate_qa_from_text(text: str, num_pairs: int = 5) -> List[Dict]:
    """
    Text'ten QA pair'leri oluştur (kendi local LLM API ile)
    """
    client = OpenAI(
        base_url=API_BASE,
        api_key=API_KEY
    )
    
    prompt = f"""Aşağıdaki metinden {num_pairs} adet yüksek kaliteli Türkçe soru-cevap çifti oluştur.
Sorular metinde geçen önemli kavramları test etmeli, cevaplar detaylı ve açıklayıcı olmalı.

Metin:
{text}

JSON formatında döndür:
{{
    "qa_pairs": [
        {{
            "question": "...",
            "answer": "..."
        }}
    ]
}}
"""
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "Sen bir eğitim verisi oluşturma uzmanısın. Yüksek kaliteli QA çiftleri oluşturursun."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=2048
    )
    
    result_text = response.choices[0].message.content
    
    # JSON parse et
    try:
        # Bazen model ``` ile wrap edebilir, temizle
        if "```json" in result_text:
            result_text = result_text.split("```json")[1].split("```")[0]
        elif "```" in result_text:
            result_text = result_text.split("```")[1].split("```")[0]
        
        data = json.loads(result_text)
        return data.get("qa_pairs", [])
    
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON parse hatası: {e}")
        print(f"Raw output:\n{result_text}")
        return []

print("✅ QA generator fonksiyonu hazır!")

In [ ]:
# Fonksiyonu test et
print("🔄 Local LLM ile QA pair'leri oluşturuluyor...\n")

qa_pairs = generate_qa_from_text(sample_text, num_pairs=3)

print(f"✅ {len(qa_pairs)} QA pair oluşturuldu!\n")
print("="*80)

for i, qa in enumerate(qa_pairs, 1):
    print(f"\n#{i}")
    print(f"❓ Soru: {qa.get('question', 'N/A')}")
    print(f"✅ Cevap: {qa.get('answer', 'N/A')}")
    print("-"*80)

### 🔧 Batch İşleme - Birden Fazla Dosya

Bir klasördeki tüm dosyaları toplu olarak işleyelim:

In [ ]:
# Birkaç örnek dosya daha ekleyelim
sample_texts = {
    "python_basics.txt": """
Python Programlama Dili

Python, Guido van Rossum tarafından 1991 yılında geliştirilmiş, yüksek seviyeli bir programlama dilidir.
Basit sözdizimi ve geniş kütüphane desteği sayesinde veri bilimi, web geliştirme ve yapay zeka 
projelerinde yaygın olarak kullanılır.

Python'da veri tipleri: int, float, str, list, dict, tuple gibi temel tipler vardır.
List comprehension, generator expression gibi özellikler kodu daha okunabilir yapar.
""",
    "rag_systems.txt": """
RAG (Retrieval Augmented Generation) Sistemleri

RAG, LLM'lerin bilgi tabanlarına erişerek daha doğru ve güncel yanıtlar vermesini sağlar.
Sistem, kullanıcı sorgusunu embedding'e dönüştürür, vektör veritabanında benzer dokümanları 
arar ve bunları LLM'e context olarak verir.

Popüler vektör veritabanları: Pinecone, Weaviate, ChromaDB, FAISS.
Embedding modelleri: OpenAI embeddings, sentence-transformers.
"""
}

# Dosyaları kaydet
for filename, content in sample_texts.items():
    filepath = INPUT_DIR / filename
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"✅ Oluşturuldu: {filename}")

print(f"\n📂 Toplam dosya sayısı: {len(list(INPUT_DIR.glob('*.txt')))}")

In [ ]:
# Tüm dosyaları toplu işle
def process_all_files(input_dir: Path, num_qa_per_file: int = 3):
    """
    Bir dizindeki tüm text dosyalarını işleyip QA pair'leri oluştur
    """
    all_qa_pairs = []
    
    for text_file in input_dir.glob("*.txt"):
        print(f"\n🔄 İşleniyor: {text_file.name}")
        
        # Dosyayı oku
        with open(text_file, "r", encoding="utf-8") as f:
            content = f.read()
        
        # QA pair'leri oluştur
        qa_pairs = generate_qa_from_text(content, num_pairs=num_qa_per_file)
        
        # Kaynak dosyayı ekle
        for qa in qa_pairs:
            qa['source_file'] = text_file.name
        
        all_qa_pairs.extend(qa_pairs)
        print(f"   ✅ {len(qa_pairs)} QA oluşturuldu")
    
    return all_qa_pairs

# Tüm dosyaları işle
print("🚀 Batch işleme başlıyor...\n")
all_qa = process_all_files(INPUT_DIR, num_qa_per_file=2)

print(f"\n✅ Toplam {len(all_qa)} QA pair oluşturuldu!")

In [ ]:
# Tüm QA'ları kaydet
batch_output = GENERATED_DIR / "batch_qa_pairs.json"
with open(batch_output, "w", encoding="utf-8") as f:
    json.dump(all_qa, f, indent=2, ensure_ascii=False)

print(f"💾 Batch QA'lar kaydedildi: {batch_output}")

# Özet göster
print(f"\n📊 Özet:")
for file in INPUT_DIR.glob("*.txt"):
    count = sum(1 for qa in all_qa if qa.get('source_file') == file.name)
    print(f"   {file.name}: {count} QA")

---

## 6️⃣ Farklı Format Örnekleri

Alpaca dışında başka formatlar da kullanabilirsiniz:

In [ ]:
# Alpaca format
def convert_to_alpaca(qa_pairs: List[Dict]) -> List[Dict]:
    """QA'ları Alpaca formatına dönüştür"""
    alpaca_data = []
    for qa in qa_pairs:
        alpaca_data.append({
            "instruction": qa.get('question', ''),
            "input": "",
            "output": qa.get('answer', '')
        })
    return alpaca_data

# ShareGPT format
def convert_to_sharegpt(qa_pairs: List[Dict]) -> List[Dict]:
    """QA'ları ShareGPT formatına dönüştür"""
    sharegpt_data = []
    for qa in qa_pairs:
        sharegpt_data.append({
            "conversations": [
                {"from": "human", "value": qa.get('question', '')},
                {"from": "gpt", "value": qa.get('answer', '')}
            ]
        })
    return sharegpt_data

# ChatML format
def convert_to_chatml(qa_pairs: List[Dict]) -> List[str]:
    """QA'ları ChatML formatına dönüştür"""
    chatml_data = []
    for qa in qa_pairs:
        chatml = f"""<|im_start|>user
{qa.get('question', '')}<|im_end|>
<|im_start|>assistant
{qa.get('answer', '')}<|im_end|>"""
        chatml_data.append(chatml)
    return chatml_data

# Dönüşümleri yap
alpaca_format = convert_to_alpaca(all_qa)
sharegpt_format = convert_to_sharegpt(all_qa)
chatml_format = convert_to_chatml(all_qa)

# Kaydet
with open(CURATED_DIR / "alpaca_format.json", "w", encoding="utf-8") as f:
    json.dump(alpaca_format, f, indent=2, ensure_ascii=False)

with open(CURATED_DIR / "sharegpt_format.json", "w", encoding="utf-8") as f:
    json.dump(sharegpt_format, f, indent=2, ensure_ascii=False)

with open(CURATED_DIR / "chatml_format.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(chatml_format))

print("✅ Tüm formatlar kaydedildi:")
print(f"   - Alpaca: {CURATED_DIR / 'alpaca_format.json'}")
print(f"   - ShareGPT: {CURATED_DIR / 'sharegpt_format.json'}")
print(f"   - ChatML: {CURATED_DIR / 'chatml_format.txt'}")

In [ ]:
# Format örneklerini göster
print("📦 Format Örnekleri:\n")
print("=" * 80)

print("\n1️⃣ Alpaca Format:")
print(json.dumps(alpaca_format[0], indent=2, ensure_ascii=False))

print("\n" + "=" * 80)
print("\n2️⃣ ShareGPT Format:")
print(json.dumps(sharegpt_format[0], indent=2, ensure_ascii=False))

print("\n" + "=" * 80)
print("\n3️⃣ ChatML Format:")
print(chatml_format[0])

---

## 7️⃣ İpuçları ve Best Practices

### 🎯 Parametre Optimizasyonu

- **chunk_size**: 1500-2500 karakter arası optimal
- **temperature**: 0.7-0.9 arası çeşitlilik için iyi
- **quality_threshold**: 7.0-8.0 arası dengeli
- **num_pairs**: Chunk başına 3-5 QA optimal

### 🚀 Performans

- Büyük dosyalar için önce küçük batch test edin
- GPU kullanın (eğer local API GPU destekliyorsa)
- Paralel işleme için birden fazla worker kullanın

### ✅ Kalite Kontrolü

- Her adımdan sonra manuel olarak birkaç örneği inceleyin
- Quality threshold'u kademeli olarak ayarlayın
- Farklı temperature değerlerini test edin

### 📁 Dosya Yönetimi

- Kaynak dosyalarınızı versiyonlayın
- Her run için timestamp ekleyin
- Büyük dataset'leri JSONL formatında saklayın (bellek tasarrufu)

---

## 8️⃣ Troubleshooting

### ❌ "Connection Error" Hatası

**Çözüm:**
- Local LLM API'nizin çalıştığından emin olun
- `API_BASE` URL'ini kontrol edin
- `curl {API_BASE}/models` ile test edin

### ❌ "Model Not Found" Hatası

**Çözüm:**
- `MODEL_NAME` değişkenini kontrol edin
- API'nizde hangi modeller var: `curl {API_BASE}/models`

### ❌ JSON Parse Hatası

**Çözüm:**
- LLM'nizin JSON üretimi zayıf olabilir
- Prompt'u daha net yapın
- Temperature'ü azaltın (0.5-0.6)
- Daha büyük/iyi model kullanın

### ❌ Düşük Kaliteli QA'lar

**Çözüm:**
- Chunk size'ı ayarlayın (çok küçük chunk → context eksikliği)
- System prompt'u iyileştirin
- Quality threshold'u yükseltin
- Daha güçlü model kullanın

---

## 9️⃣ Özet

### ✅ Bu Notebook'ta Öğrendikleriniz:

1. ✅ Synthetic Data Kit kurulumu
2. ✅ Kendi local LLM API'nizi yapılandırma (vLLM olmadan!)
3. ✅ 4 adımlı pipeline: Ingest → Create → Curate → Save
4. ✅ Programmatik kullanım (Python API)
5. ✅ Batch işleme (birden fazla dosya)
6. ✅ Farklı format dönüşümleri (Alpaca, ShareGPT, ChatML)
7. ✅ Best practices ve troubleshooting

### 🎯 Sonraki Adımlar:

- Kendi dokümanlarınızı `input/` dizinine koyun
- Farklı QA tiplerini deneyin (CoT, Summary)
- Fine-tuning için oluşturduğunuz veriyi kullanın
- Kalite kontrolü yapın ve iyileştirin

### 📚 Kaynaklar:

- [Synthetic Data Kit GitHub](https://github.com/meta-llama/synthetic-data-kit)
- [Meta Llama Models](https://github.com/meta-llama)
- [OpenAI API Docs](https://platform.openai.com/docs/api-reference)

---

**🎉 Başarılar! Artık kendi local modelinizle sentetik veri üretebilirsiniz!**